<a href="https://colab.research.google.com/github/ITA-LOW/projeto_senex/blob/main/Projeto_Senex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install pySUS==0.5.8
    !pip install geopandas
    !pip install dython

#assim que baixar é recomendável dar um "restart runtime" que tem logo abaixo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.8 MB/s 
     |████████████████████████████████| 98 kB 7.5 MB/s 
     |████████████████████████████████| 1.5 MB 58.5 MB/s 
     |████████████████████████████████| 1.6 MB 54.2 MB/s 
  Created wheel for pySUS: filename=PySUS-0.5.8-cp37-cp37m-linux_x86_64.whl size=85921 sha256=8aa6da35331771edd0fe3625678e82d89d6e0efea1d017fa4c3d81caa88b770c
  Stored in directory: /root/.cache/pip/wheels/b3/4c/85/99b0a0f6d313882dd493d9be6aa6c619e0797b57ef23aee3ce
Successfully built pySUS
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 15.4 MB/s 
     |████████████████████████████████| 6.3 MB 32.4 MB/s 
     |████████████████████████████████| 16.7 MB 387 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████

In [ ]:
from pysus.online_data import SIM
from pysus.preprocessing.decoders import decodifica_idade_SIM, translate_variables_SIM
import pandas as pd
import numpy as np

In [ ]:
def dwdados(ano, estado, conv_cat=True, create_datetime=True, delete_old_datetime=True):
    df = SIM.download(estado, ano)
    
    df.IDADE = pd.to_numeric(df.IDADE)
    
    if (create_datetime==True):
        df['DATETIME'] = pd.to_datetime(df['DTOBITO']+' '+df['HORAOBITO'], format="%d%m%Y %H%M", errors='coerce')
        
    if (delete_old_datetime==True):
        df=df.drop(columns=['DTOBITO', 'HORAOBITO'])

     
    return df

def trocar_indice(serie, dicionario):
    idx = serie.index
    nidx = pd.Series(idx).map(dicionario, na_action='ignore')
    serie.index = nidx
    return serie

def filtro_cid (df, lista_cid, colunas=['LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII']):
    # search_values = ['J960'], 
    return df[df[colunas].stack().str.contains('|'.join(lista_cid),case=False,na=False).groupby(level=0).any()]

def converter_idade(df):
    ndf = df.copy()
    #df['IDADE'] = df['IDADE'].apply(lambda x: x - 400 if x > 400 else 0 )
    ndf['IDADE'] = ndf['IDADE'].apply(lambda x: x - 400 if x > 400 else 0 )
    ndf['IDADE'] = ndf['IDADE'].apply(lambda x: x if x != 599 else np.nan )
    #idade=(df[(df['IDADE']>400) & (df['IDADE']<550)]['IDADE']-400)    
    return ndf

In [ ]:
estados =['SC'] #colocar os estados
anos = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020] #colocar os anos
banco={}
for y in anos:
 for uf in estados: 
  banco[uf, y] = dwdados(y, uf, create_datetime=False, delete_old_datetime=False)

dados = pd.concat({ k: pd.DataFrame.from_dict(v) 
for k, v in banco.items() }, axis=0).reset_index()

dados.to_csv('dados_2011_2020.csv')

In [ ]:
dados.rename(columns={'level_0': 'ESTADO', 'level_1': 'ANO'}, inplace=True) #renomeando colunas
dados.columns

Index(['ESTADO', 'ANO', 'level_2', 'CONTADOR', 'ORIGEM', 'TIPOBITO', 'DTOBITO',
       'HORAOBITO', 'NATURAL', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV',
       'ESC', 'OCUP', 'CODMUNRES', 'CODBAIRES', 'LOCOCOR', 'CODESTAB',
       'CODMUNOCOR', 'CODBAIOCOR', 'IDADEMAE', 'ESCMAE', 'OCUPMAE',
       'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'GESTACAO', 'PARTO',
       'OBITOPARTO', 'PESO', 'OBITOGRAV', 'OBITOPUERP', 'ASSISTMED', 'EXAME',
       'CIRURGIA', 'NECROPSIA', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD',
       'LINHAII', 'CAUSABAS', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE',
       'TPPOS', 'DTINVESTIG', 'CAUSABAS_O', 'DTCADASTRO', 'ATESTANTE',
       'FONTEINV', 'DTRECEBIM', 'UFINFORM', 'CB_PRE', 'MORTEPARTO', 'DTCADINF',
       'TPOBITOCOR', 'DTCADINV', 'COMUNSVOIM', 'DTRECORIG', 'DTRECORIGA',
       'CAUSAMAT', 'ESC2010', 'ESCMAE2010', 'STDOEPIDEM', 'STDONOVA',
       'SEMAGESTAC', 'TPMORTEOCO', 'DIFDATA', 'DTCONCASO', 'NUDIASOBIN',
       'SERIESCFAL', 'SERIESCMAE', 'CO